<a href="https://colab.research.google.com/github/maksim-mihtech/gdp-dashboard/blob/main/streamlit2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q
!pip install graphistry -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.1/265.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 14.7 MB/s eta 0:00:00


In [21]:
#!wget -q -O - ipv4.icanhazip.com
## For multi page app we need to have all pages(page1.py, page2.py and etc) under the same folde(/content/streamlit)
#!mkdir /content/streamlit

In [ ]:
import pandas as pd
import numpy as np
import datetime

df = pd.read_json('/content/sample_data/gl.01.json', lines=True)

df.head()
#df.info()

,@timestamp,IN_SRC_MAC,OUT_DST_MAC,IPV4_SRC_ADDR,SRC_ADDR_LOCAL,SRC_ADDR_BLACKLISTED,SRC_ADDR_SERVICES,SRC_NAME,IPV4_DST_ADDR,DST_ADDR_LOCAL,...,SERVER_NW_LATENCY_MS,TLS_SERVER_NAME,IPV6_SRC_ADDR,IPV6_DST_ADDR,TLS_CERT_CN,HTTP_HOST,HTTP_URL,HTTP_USER_AGENT,HTTP_METHOD,HTTP_RET_CODE
0,2024-07-16T13:00:30.0Z,30:07:4D:5B:C8:DE,F6:0F:24:92:6A:64,192.168.2.5,False,False,0,,192.168.2.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-07-16T13:00:30.0Z,30:07:4D:5B:C8:DE,F6:0F:24:92:6A:64,192.168.2.5,False,False,0,,212.199.202.35,False,...,5.795,a.postrelease.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-07-16T13:00:30.0Z,F4:0F:24:29:A8:E0,01:00:5E:00:00:16,0.0.0.0,True,False,0,NoIP,224.0.0.22,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-07-16T13:00:30.0Z,30:07:4D:5B:C8:DE,F6:0F:24:92:6A:64,192.168.2.5,False,False,0,,72.34.250.75,False,...,110.384,sync.go.sonobi.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-07-16T13:00:30.0Z,30:07:4D:5B:C8:DE,F6:0F:24:92:6A:64,192.168.2.5,False,False,0,,64.74.236.95,False,...,106.140,mcdp-chidc2.outbrain.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
def accumulate_df(df):
    # assign field/column names to variables
    f_timestamp = 'FIRST_SWITCHED'
    f_src_ip = 'IPV4_SRC_ADDR'
    f_dst_ip = 'IPV4_DST_ADDR'
    f_L7_proto = 'L7_PROTO_NAME'
    ##Upload bytes
    f_sent_bytes = 'IN_BYTES'
    #Download
    f_received_bytes = 'OUT_BYTES'

    f_delimiter = '\t'
    f_dst_name = 'DST_NAME'

    ## Every session has Upload/Download data and duration
    #df['data'] = df['IN_BYTES'] + df['OUT_BYTES']
    #f_data = 'data'
    df['dur'] = df['LAST_SWITCHED'] - df['FIRST_SWITCHED']
    f_duration = 'dur'

    columns_to_groupby = [f_src_ip, f_dst_ip, f_L7_proto]
    ip_df = df.groupby(columns_to_groupby).agg(list)
    ip_df.reset_index(inplace=True)
    ip_df['conn_count'] = ip_df[f_timestamp].apply(lambda x: len(x))
    ## Filter out sessions with conn_count greate than 20
    #ip_df = ip_df.loc[ip_df['conn_count'] > 20]
    ip_df['download'] = ip_df[f_received_bytes].apply(lambda x: pd.Series(x).sum())
    ip_df['upload'] = ip_df[f_sent_bytes].apply(lambda x: pd.Series(x).sum())
    ip_df['dur_total'] = ip_df[f_duration].apply(lambda x: pd.Series(x).sum())

    #Gap is a difference between Upload and Download, it's a feature of active exfiltration is destination is uknown and unattributable
    ip_df['gap'] = ip_df['upload'] - ip_df['download']

    ## Look for accumulated events with gap greater that 100000(It's low sensitivity and for big dataset it should be increased)
    ip_df = ip_df.loc[ip_df['gap'] > 0]
    ip_df = ip_df.head(10)

    return ip_df

In [20]:
%%writefile app.py

import streamlit as st
import pandas as pd
import altair as alt
import plotly.express as px
import graphistry

#############################################################################################
def get_name_type_first_row(series):
    series_nona = series.dropna()
    if len(series_nona) == 0:
        return np.nan
    else:
        return series.dropna().iloc[0]

# function to get unique values
def list_unique(list1):
    unique_list = pd.Series(list1).drop_duplicates().tolist()
    return unique_list

def accumulate_df(df):
    # assign field/column names to variables
    f_timestamp = 'FIRST_SWITCHED'
    f_src_ip = 'IPV4_SRC_ADDR'
    f_dst_ip = 'IPV4_DST_ADDR'
    f_L7_proto = 'L7_PROTO_NAME'
    ##Upload bytes
    f_sent_bytes = 'IN_BYTES'
    #Download
    f_received_bytes = 'OUT_BYTES'

    f_delimiter = '\t'
    f_dst_name = 'DST_NAME'

    ## Every session has Upload/Download data and duration
    #df['data'] = df['IN_BYTES'] + df['OUT_BYTES']
    #f_data = 'data'
    df['dur'] = df['LAST_SWITCHED'] - df['FIRST_SWITCHED']
    f_duration = 'dur'

    if 'TLS_CERT_CN' in df.columns:
      df['DST_NAME'] = df['TLS_CERT_CN']

    columns_to_groupby = [f_src_ip, f_dst_ip, f_L7_proto]
    ip_df = df.groupby(columns_to_groupby).agg(list)
    ip_df.reset_index(inplace=True)
    ip_df['conn_count'] = ip_df[f_timestamp].apply(lambda x: len(x))
    ## Filter out sessions with conn_count greate than 20
    #ip_df = ip_df.loc[ip_df['conn_count'] > 20]
    ip_df['DST_NAME'] = ip_df['DST_NAME'].apply(list_unique)


    ip_df['download'] = ip_df[f_received_bytes].apply(lambda x: pd.Series(x).sum())
    ip_df['upload'] = ip_df[f_sent_bytes].apply(lambda x: pd.Series(x).sum())
    ip_df['dur_total'] = ip_df[f_duration].apply(lambda x: pd.Series(x).sum())

    #Gap is a difference between Upload and Download, it's a feature of active exfiltration is destination is uknown and unattributable
    ip_df['gap'] = ip_df['upload'] - ip_df['download']

    ## Look for accumulated events with gap greater that 100000(It's low sensitivity and for big dataset it should be increased)
    ip_df = ip_df.loc[ip_df['gap'] > 0]
    ip_df = ip_df.head(10)


    columns_to_filter = [f_src_ip,f_dst_ip,f_L7_proto,'upload', 'download', 'DST_NAME','conn_count']
    ip_df = ip_df.loc[:,columns_to_filter]

    ## Highest gap on top
    ip_df= ip_df.sort_values('upload', ascending=False)

    return ip_df
#############################################################################################

graphistry.register(api=3, username='mihtech', password='Lena417733')

#st.set_page_config(
#    page_title="AlphaCube Dashboard",
#    layout="wide",
#    initial_sidebar_state="expanded")


with st.sidebar:
  st.set_page_config(page_title="Ask your JSON", layout="wide")
  st.header("Ask your JSON")
  st.title("AlphaCube Dashboard")

  json_file = st.file_uploader("Upload a JSON file", type="json")

if json_file is not None:
  df = pd.read_json(json_file, lines=True)
  #df = pd.read_csv(csv_file, delimiter=',')

  with st.sidebar:
    #print(f"Number of distinct IPV4_SRC_ADDR: {df['IPV4_SRC_ADDR'].nunique()}")
    #df['IPV4_SRC_ADDR'].nunique()
    #st.write(f"Number of distinct IPV4_SRC_ADDR: {df['IPV4_SRC_ADDR'].nunique()}")
    df2 = df.copy()
    df3 = df.copy()
    df2_grouped = df2.groupby(['IPV4_SRC_ADDR']).size().reset_index(name='Size')
    st.write(df2_grouped)

    df3 = df3[df3['HTTP_USER_AGENT'].notnull()]
    if not df3.empty:
      df3_grouped = df3.groupby(['IPV4_SRC_ADDR','HTTP_USER_AGENT']).size().reset_index(name='Size')
      st.write(df3_grouped)
    else:
      st.write('No HTTP_USER_AGENT found')


    # Add a multiselect widget to select rows based on the index
    #selected_indices = st.multiselect('Select rows:', result.index)
    # Subset the dataframe with the selected indices
    #selected_rows = result.loc[selected_indices]
    # Display the selected data
    #st.write('Selected Row IPs:')
    #st.write(selected_rows['IPV4_SRC_ADDR'])



#df = pd.read_json('/content/sample_data/gl.01.json', lines=True)

  tab1, tab2, tab3 = st.tabs(["IP Address", "Sevice-based", "More"])



##
  with tab1:
      st.header("IP-based")
      ## By defauly 'Dataframe Mutability' effects 'df' and 'ip_df' therefore we create copy
      df1 = df.copy()
      ## Accumulate records
      ip_df = accumulate_df(df1)
      st.table(ip_df)

## Graph of Accumulated Top 100 sessions(by the count)
#df_graph = df3.head(100)
      df_graph=ip_df.head(100)
      hg = graphistry.hypergraph(
        df_graph,
        entity_types=['IPV4_SRC_ADDR','IPV4_DST_ADDR','DST_NAME'],
        direct=True,
        opts={
            'EDGES': {
              'IPV4_SRC_ADDR': ['IPV4_DST_ADDR'],
              'IPV4_DST_ADDR': ['DST_NAME'],
               }
        })

      RENDER = False
      link = hg['graph'].plot(render=RENDER)
      st.components.v1.iframe(link, height=500, scrolling=True)
##end of tab 1

  with tab2:
    st.header("IP-based")
  ## By defauly 'Dataframe Mutability' effects 'df' and 'ip_df' therefore we create copy
    df4 = df.copy()
    st.table(df4.head(10))

Overwriting app.py


In [3]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [21]:
!wget -q -O - ipv4.icanhazip.com

34.168.100.201


In [22]:
#!streamlit run app.py & npx localtunnel --port 8501
!streamlit run app.py &>/content/logs.txt &

## Multipage App structure
#!streamlit run streamlit/streamlit_app.py &>/content/logs.txt &

In [23]:
!npx localtunnel --port 8501

your url is: https://neat-shrimps-talk.loca.lt
^C
